In [ ]:
from google.colab import files
uploaded = files.upload()

Saving plate_number.zip to plate_number.zip


In [ ]:
import zipfile
import os

# Replace 'plates.zip' with your uploaded filename
zip_filename = 'plate_number.zip'
extract_path = 'plate_number'

with zipfile.ZipFile(zip_filename, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

print("✅ Images extracted to:", extract_path)

✅ Images extracted to: plate_number


In [ ]:
!pip install easyocr opencv-python-headless pandas

In [ ]:
!rm -rf yolov5  # Remove if exists
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 17485, done.
remote: Counting objects: 100% (111/111), done.
remote: Compressing objects: 100% (80/80), done.
remote: Total 17485 (delta 81), reused 31 (delta 31), pack-reused 17374 (from 3)
Receiving objects: 100% (17485/17485), 16.39 MiB | 22.80 MiB/s, done.
Resolving deltas: 100% (11986/11986), done.


In [ ]:
%cd yolov5
!pip install -qr requirements.txt
%cd ..

/content/yolov5
/content


In [ ]:
import torch
import easyocr
import cv2
import os
import pandas as pd
from matplotlib import pyplot as plt
import re

# Setup
image_folder = '/content/plate_number/plate_number'
model_path = 'best.pt'

# Load YOLOv5 model
model = torch.hub.load('yolov5', 'custom', path=model_path, source='local')
model.conf = 0.4

# OCR Reader
reader = easyocr.Reader(['en'], gpu=True)

# Special plate prefixes (Malaysian unique plates)
special_prefixes = {'GOLD', 'UKM', 'UUM', 'UM', 'USM', 'IIUM', 'ASEAN', 'G1M', 'UNISZA', 'UMT', 'UMK', 'UMS', 'UNIMAS'}

# Output list
results = []

# Loop through images
for filename in sorted(os.listdir(image_folder)):
    if filename.lower().endswith(('.jpg', '.jpeg', '.png')):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)

        # Preprocess image: grayscale + threshold
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        _, thresh = cv2.threshold(gray, 150, 255, cv2.THRESH_BINARY)

        # Run YOLOv5 detection on the preprocessed image
        detections = model(thresh)
        df = detections.pandas().xyxy[0]

        plate_text = 'Not Detected'
        confidence_score = 0.0

        if len(df) > 0:
            # Assume the most confident detection
            top_row = df.iloc[0]
            xmin, ymin, xmax, ymax = map(int, [top_row['xmin'], top_row['ymin'], top_row['xmax'], top_row['ymax']])
            plate_crop = img[ymin:ymax, xmin:xmax]

            # OCR on cropped plate
            ocr_result = reader.readtext(plate_crop)

            if ocr_result:
                # Combine detected text lines with high confidence
                raw_text = ' '.join([result[1] for result in ocr_result if result[2] > 0.5])
                raw_text = raw_text.upper()
                raw_text = ''.join(filter(str.isalnum, raw_text))  # Remove non-alphanumeric

                # Correct common misreads
                corrections = {'S': '5', 'O': '0', 'I': '1', 'Z': '2', 'B': '8'}
                corrected_text = ''.join([corrections.get(char, char) for char in raw_text])

                # Insert space between alphabet and number transitions
                formatted_text = re.sub(r'([A-Za-z])(\d)', r'\1 \2', corrected_text)
                formatted_text = re.sub(r'(\d)([A-Za-z])', r'\1 \2', formatted_text)

                # Apply special prefix priority logic (optional bump to confidence)
                for prefix in special_prefixes:
                    if formatted_text.startswith(prefix):
                        confidence_score += 0.05
                        break

                plate_text = formatted_text.strip()
                confidence_score = max([result[2] for result in ocr_result])

        results.append({
            'Image': filename,
            'Detected Plate': plate_text,
            'Confidence': round(confidence_score, 2)
        })

# Show results in table
df_result = pd.DataFrame(results)
df_result


YOLOv5 🚀 v7.0-418-ga493afe1 Python-3.11.12 torch-2.6.0+cu124 CPU



Exception: [Errno 2] No such file or directory: 'best.pt'. Cache may be out of date, try `force_reload=True` or see https://docs.ultralytics.com/yolov5/tutorials/pytorch_hub_model_loading for help.

In [ ]:
# Create results DataFrame
df_result = pd.DataFrame(results)

# Define a function to count digits
def digit_count(s):
    return len(re.findall(r'\d', s))

# Filter out:
# 1. 'Not Detected'
# 2. Blank strings
# 3. Plates with no digits
# 4. Plates that start with a digit
# 5. Plates that have more than 4 digits
df_result = df_result[
    (df_result['Detected Plate'].str.strip() != '') &
    (df_result['Detected Plate'].str.upper() != 'NOT DETECTED') &
    (df_result['Detected Plate'].str.contains(r'\d')) &
    (~df_result['Detected Plate'].str.match(r'^\d')) &
    (df_result['Detected Plate'].apply(digit_count) <= 4)
].reset_index(drop=True)

# Display the cleaned result
df_result


NameError: name 'results' is not defined